## Imports

In [84]:
import numpy as np
from sklearn.linear_model import LinearRegression

from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

## Higher Order functions

In [85]:
# series of functions performing exponentiation to given power
def square(x):
    return x**2

def cube(x):
    return x**3

# make exponent and explicit argument
def generic_exp(x,exponent): # requires specify the exponent everytime you call the function
    return x**exponent



We need higher order function(function builder) to build functions (square,cube ...)

In [86]:
def skeleton_exponentiation(x):
    return x**exponent

In [87]:
# skeleton_exponentiation(2)

<span style="color:red">---------------------------------------------------------------------------</span>  
<span style="color:red">NameError</span>                                 Traceback (most recent call last)  
Cell In[9], line 1  
<span style="color:green">----> 1 skeleton_exponentiation(2)</span>  
  
Cell In[8], line 2  
      1 def skeleton_exponentiation(x):  
<span style="color:green">----> 2     return x**exponent</span>  
  
<span style="color:red">NameError</span>: name 'exponent' is not defined  

In [88]:
def exponentiation_builder(exponent):
    def skeleton_exponentiation(x):
        return x**exponent

    return skeleton_exponentiation

In [89]:
returned_function=exponentiation_builder(2)
returned_function

<function __main__.exponentiation_builder.<locals>.skeleton_exponentiation(x)>

In [90]:
exponentiation_builder(2)(4)

16

In [91]:
returned_function(5)

25

In [92]:
cube=exponentiation_builder(3)
cube(5)

125

In [116]:
# Helper functiom 1
def make_train_step(model,loss_fn,optimizer): 
    # Build a function that perform a step in the train loop
    def perform_train_step_fn(x,y):

        # set model to train state
        model.train()

        # forward pass
        y_hat=model(x)

        # compute loss
        loss=loss_fn(y_hat,y)

        # compute gradients
        loss.backward()

        # update parameters
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()

    # return the function that will be called inside the train loop
    return perform_train_step_fn 

Run: Data preperation 

In [94]:
%run -i data_generation/simple_linear_regression.py

<Figure size 640x480 with 0 Axes>

In [95]:
%run -i data_preparation/v0.py

In [117]:
%%writefile model_configuration/v1.py

device='cuda' if torch.cuda.is_available() else 'cpu'
lr=0.01

torch.manual_seed(42)
# create a model and send it to device
model=nn.Sequential(nn.Linear(1,1)).to(device)

# define SGD optmizer
optimizer=optim.SGD(model.parameters(),lr=lr)

# define loss function
loss_fn=nn.MSELoss(reduction='mean')

# create train step function
train_step_fn=make_train_step(model,loss_fn,optimizer)


Overwriting model_configuration/v1.py


In [118]:
%run -i model_configuration/v1.py

In [98]:
train_step_fn

<function __main__.make_train_step.<locals>.perform_train_step_fn(x, y)>

In [119]:
%%writefile model_training/v1.py

n_epochs=10000
losses=[]

for epoch in tqdm(range(n_epochs)):

    # perform train step and return corresponding loss
    loss=train_step_fn(X_train_tensor,y_train_tensor) # perform one training step
    losses.append(loss) # keep track of loss

Overwriting model_training/v1.py


In [120]:
%run -i model_training/v1.py

  0%|          | 0/10000 [00:00<?, ?it/s]

In [121]:
model.state_dict()

OrderedDict([('0.weight', tensor([[1.9689]])), ('0.bias', tensor([1.0236]))])

## Dataset

In [129]:
class CustomDataset(Dataset):
    # takes whatever arguments that need to create a list of tuples
    def __init__(self,x_tensor,y_tensor):
        self.x=x_tensor
        self.y=y_tensor

    # return a tuple corresponding to the index
    def __getitem__(self,index):
        return (self.x[index],self.y[index])

    # return the size of the dataset
    def __len__(self):
        return len(self.x)

X_train_tensor=torch.as_tensor(X_train).float()
y_train_tensor=torch.as_tensor(y_train).float()

train_data=CustomDataset(X_train_tensor,y_train_tensor)
print(train_data[[1,2,4]])

(tensor([[0.0636],
        [0.8631],
        [0.7320]]), tensor([[1.1928],
        [2.9128],
        [2.4732]]))
